In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')
from multiprocessing import Pool, cpu_count, RLock
from scipy import stats
from statsmodels.tsa.stattools import adfuller

from financial_data_structure import resample_bar

In [3]:
data = pd.read_csv(
    '~/crypto_data/exchange_data/combined/BTCSPOT_3600.csv', index_col=0, parse_dates=True)
data.shape

(40616, 5)

In [10]:
def get_weights(d, size):
    w = [1]
    for k in range(1, size):
        w_ = -w[-1] / k * (d - k + 1)
        w.append(w_)
    return np.array(w[::-1])


def frac_diff_ffd(series, d, thres=1e-5):
    w = get_weights(d, series.shape[0])
    w = w[np.abs(w) > thres]
    result = series.rolling(len(w)).apply(lambda x: np.dot(w, x)).dropna()
    return result


def get_diff_factor(series):
    d = 0.05
    result = {}
    while d != 1:
        print(f'Processing {d}')
        # print(f'Processing {d}', end='\r')
        s = frac_diff_ffd(series, d)
        result[d] = s.apply(lambda x: adfuller(x)[1])
        d += 0.05
    return pd.DataFrame(result, columns=series.columns)


get_diff_factor(data)

KeyboardInterrupt: 